In [13]:
%%writefile src/OrderEvent.sol
//SPDX-License-Identifier: GPL-3.0-or-later
pragma solidity ^0.8.0;

contract OrderEvent {
    uint unitPrice = 10;
    //event OrderLog(string);
    address private owner;
    //event OrderLog(bytes2 _itemId, uint _value);
    //event OrderLog(uint256 timestamp);
    event OrderLog(address indexed _from, bytes2 _itemId, uint indexed _value, string _shipping);

    function order(bytes2 _itemId, uint quantity, string memory where) public payable {
        //uint256 orderTime = now;
        //uint256 orderTime = block.timestamp;
        uint256 orderAmount = quantity * unitPrice;
        string memory _shipping = where;
        require(msg.value == orderAmount);
        //emit OrderLog("Ordered");
        //emit OrderLog(orderTime);
        emit OrderLog(msg.sender, _itemId, msg.value, _shipping);
    }

    function getBalance()view public isOwner returns(uint){
        return address(this).balance;
    }
    modifier isOwner(){
        if(msg.sender == owner){
            revert();
        }
        _;
    }
}

Overwriting src/OrderEvent.sol


In [14]:
!solc-windows.exe --optimize --combined-json abi,bin src/OrderEvent.sol > src/OrderEvent.json

In [15]:
%%writefile src/OrderEventDeploy.js
var Web3 = require('web3');
var _abiBinJson = require('./OrderEvent.json');      //importing a javascript file

var web3 = new Web3(new Web3.providers.HttpProvider("http://localhost:8345"));

contractName=Object.keys(_abiBinJson.contracts); // reading ['src/OrderEvent.sol:OrderEvent']
console.log("- contract name: ", contractName);
_abiArray=_abiBinJson.contracts[contractName].abi;
//_abiArray=JSON.parse(JSON.stringify(_abi));
//_abiArray=JSON.parse(_abi);      //JSON parsing needed!! //SyntaxError: Unexpected token o in JSON at position 1
_bin=_abiBinJson.contracts[contractName].bin;

//console.log("- ABI: " + _abiArray);
//console.log("- Bytecode: " + _bin);

async function deploy() {
    const accounts = await web3.eth.getAccounts();
    console.log("Deploying the contract from " + accounts[0]);
    var deployed = await new web3.eth.Contract(_abiArray)
        .deploy({data: "0x"+_bin})
        .send({from: accounts[0], gas: 259210}, function(err, transactionHash) {
                if(!err) console.log("hash: " + transactionHash); 
        })
        //.then(function(newContractInstance){
        //    console.log(newContractInstance)
        //});
    console.log("---> The contract deployed to: " + deployed.options.address)
}
deploy()

Overwriting src/OrderEventDeploy.js


In [16]:
!node src/OrderEventDeploy.js

- contract name:  [ 'src/OrderEvent.sol:OrderEvent' ]
Deploying the contract from 0xd7C2230AF5F017D7F5F5df5535EC2Dbeec199C01
hash: 0x3742455df4b6a8d192e02ec9165579f57bbfb110b8bdf6309deb30e2f1cedbf2
---> The contract deployed to: 0x4F02875aC9f24fC80A9Bf113Cf66d67D3Ec5505c


In [5]:
%%writefile src/OrderEventUse.js
var Web3=require('web3');
var fs = require('fs');
var _abiBinJson = require('./OrderEvent.json');      //importing a javascript file

//var web3 = new Web3(new Web3.providers.HttpProvider("http://localhost:8345"));
//var web3 = new Web3(new Web3.providers.WebsocketProvider("http://localhost:8345"));
var web3 = new Web3(new Web3.providers.WebsocketProvider("ws://localhost:8345"));
contractName=Object.keys(_abiBinJson.contracts); // reading ['src/OrderEvent.sol:OrderEvent']
//console.log("- contract name: ", contractName); //or console.log(contractName[0]);
_abiArray=_abiBinJson.contracts[contractName].abi; //use just as read from file
//_abiArray=JSON.parse(JSON.stringify(_abi));
//_abiArray=JSON.parse(_abi);      //JSON parsing needed!!
//_bin=_abiBinJson.contracts[contractName].bin;
console.log("- ABI: " + _abiArray);
//console.log("- Bytecode: " + _bin);
//var _test = new web3.eth.Contract(_abiArray,"0x4F02875aC9f24fC80A9Bf113Cf66d67D3Ec5505c");
//var event = _test.events.MyLog({fromBlock: 0}, function (error, result) {
//    if (!error) {
//        console.log("Event fired: " + JSON.stringify(result.returnValues));
//    }
//});

async function doIt() {
   var _order = new web3.eth.Contract(_abiArray, '0x4F02875aC9f24fC80A9Bf113Cf66d67D3Ec5505c');
   var event = _order.events.OrderLog({fromBlock: 0}, function (error, result) {
    if (!error) {
        log = JSON.stringify(result.returnValues);
        console.log("Event fired: " + log);
        fs.appendFile("src/OrderEvent.txt", log, "utf-8", function(e) {
            if(!e) {
                console.log(">> Writing to file");
            }
        });
    }
});
    
    const accounts = await web3.eth.getAccounts();
    console.log("Account: " + accounts[0]);
   
    var value;
    // this will fire an event
    my = await _order.methods.order("0x1111", 3, "20 gil Hongji-dong Jongro-gu Seoul")
        .send({from: accounts[0], gas: 100000, value:30})
        //.then(function(my) {console.log("---> MyFunction called " + JSON.stringify(my.events.OrderLog.returnValues));});
    //console.log("---> MyFunction called " + JSON.stringify(my.events.OrderLog.returnValues));
    // this will fire another event
    
    my = await _order.methods.order("0x1111", 5, "20 gil Hongji-dong Jongro-gu Seoul")
        .send({from: accounts[0], gas: 100000, value:50})
    
    my = await _order.methods.order("0x1111", 20, "20 gil Hongji-dong Jongro-gu Seoul")
        .send({from: accounts[0], gas: 100000, value:200})

    process.exit(1); //force exit to disconnect websocket
}

doIt()

Overwriting src/OrderEventUse.js


In [6]:
!node src/OrderEventUse.js

- ABI: [object Object],[object Object],[object Object]

C:\Users\Administrator\Code\201810825\node_modules\web3-core-helpers\src\errors.js:32
        return new Error('Invalid number of parameters for "'+ method +'". Got '+ got +' expected '+ expected +'!');
               ^

Error: Invalid number of parameters for "order". Got 3 expected 4!
    at Object.InvalidNumberOfParams (C:\Users\Administrator\Code\201810825\node_modules\web3-core-helpers\src\errors.js:32:16)
    at Object._createTxObject (C:\Users\Administrator\Code\201810825\node_modules\web3-eth-contract\src\index.js:791:22)
    at doIt (C:\Users\Administrator\Code\201810825\src\OrderEventUse.js:42:31)
    at processTicksAndRejections (node:internal/process/task_queues:96:5)



Account: 0x35cB93f50fe90B71d42f4509b1057030f8Ad65ec


In [19]:
!type src\OrderEvent.txt

{"0":"0xd7C2230AF5F017D7F5F5df5535EC2Dbeec199C01","1":"0x1111","2":"30","3":"20 gil Hongji-dong Jongro-gu Seoul","_from":"0xd7C2230AF5F017D7F5F5df5535EC2Dbeec199C01","_itemId":"0x1111","_value":"30","_shipping":"20 gil Hongji-dong Jongro-gu Seoul"}{"0":"0xd7C2230AF5F017D7F5F5df5535EC2Dbeec199C01","1":"0x1111","2":"50","3":"20 gil Hongji-dong Jongro-gu Seoul","_from":"0xd7C2230AF5F017D7F5F5df5535EC2Dbeec199C01","_itemId":"0x1111","_value":"50","_shipping":"20 gil Hongji-dong Jongro-gu Seoul"}{"0":"0xd7C2230AF5F017D7F5F5df5535EC2Dbeec199C01","1":"0x1111","2":"200","3":"20 gil Hongji-dong Jongro-gu Seoul","_from":"0xd7C2230AF5F017D7F5F5df5535EC2Dbeec199C01","_itemId":"0x1111","_value":"200","_shipping":"20 gil Hongji-dong Jongro-gu Seoul"}
